<a href="https://colab.research.google.com/github/FFI-Vietnam/camtrap-tools/blob/main/MegaDetector/confusion_matrix/02_create_confusion_matrix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [108]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import json
import requests
import os

In [97]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [148]:
# read ground truth table
root = '/content/drive/'
ground_truth_folder = 'My Drive/FFI/MegaDetector Test/ground_truths'
ground_truth_name = 'ground-truth-table_Kon-Plong.csv'
ground_truth_path = os.path.join(root, ground_truth_folder, ground_truth_name)

ground_truth_table = pd.read_csv(ground_truth_path)
ground_truth_table.sample(5)

,FileName,CatalogSets,species_common_name
60028,kpc314__ct68908__2019-10-14__03-45-10(1)__Ferr...,"Species|Ferret-badger (unknown), Observer|An N...",Ferret-badger
67894,336__69025__2019-11-20__21-10-30(2).JPG,Observer|Tam,NaN
36016,253__67027__2019-05-04__20-52-13(1).JPG,NaN,NaN
68284,348__69010__2019-06-28__12-23-34(1).JPG,NaN,NaN
54310,kpc296__ct67011__2019-11-16__18-31-51(1)__Blan...,Observer|An Nguyen,NaN


In [149]:
# read MegaDetector result

result_folder = 'My Drive/FFI/MegaDetector Test/'
result_name = 'MegaDetector_result_2021-08-27.json'
result_path = os.path.join(root, result_folder, result_name)

with open(result_path) as f:
  mega_result = json.load(f)

# print sample result
for i in range(5):
  print(mega_result['images'][i])

{'file': '/content/drive/My Drive/Camera Trap Data/Kon Plong/2019-2020 Survey/Processed images/Final/100/67096/100__67096__2019-04-27__12-50-24(2).JPG', 'max_detection_conf': 0.922, 'detections': [{'category': '1', 'conf': 0.922, 'bbox': [0.5833, 0.9097, 0.4083, 0.09005]}]}
{'file': '/content/drive/My Drive/Camera Trap Data/Kon Plong/2019-2020 Survey/Processed images/Final/100/67096/100__67096__2019-04-24__09-55-44(1).JPG', 'max_detection_conf': 0.681, 'detections': [{'category': '1', 'conf': 0.681, 'bbox': [0.5914, 0.9117, 0.3952, 0.08826]}, {'category': '1', 'conf': 0.126, 'bbox': [0.7356, 0.917, 0.2506, 0.08295]}]}
{'file': '/content/drive/My Drive/Camera Trap Data/Kon Plong/2019-2020 Survey/Processed images/Final/100/67096/100__67096__2019-04-22__15-14-47(2).JPG', 'max_detection_conf': 0.999, 'detections': [{'category': '2', 'conf': 0.999, 'bbox': [0.01608, 0.04712, 0.7686, 0.9528]}, {'category': '2', 'conf': 0.288, 'bbox': [0.004918, 0.3709, 0.3315, 0.6175]}, {'category': '2', 'co

In [150]:
# read taxon_match_table
root = '/content/drive/'
taxon_match_folder = 'My Drive/FFI/MegaDetector Test/'
taxon_match_name = 'taxon_match_table.csv'
taxon_match_path = os.path.join(root, taxon_match_folder, taxon_match_name)

taxon_match_table = pd.read_csv(taxon_match_path)
taxon_match_table.sample(5)

,id,class,order,family,genus,species,authority,commonNameEnglish,taxonomyType,uniqueIdentifier,FFI_species_name
20,2021559,Insecta,Lepidoptera,NaN,NaN,NaN,NaN,Butterflies and Moths,biological,4a7126ec-2b0b-426d-bafd-356219eb487a,Moth
46,2015156,Aves,Passeriformes,Muscicapidae,Kittacincla,malabarica,"Scopoli, 1788",White-rumped Shama,biological,aa8579dc-5b25-4347-b1ac-31668aede87b,White-rumped Shama
125,2008859,Aves,Accipitriformes,Accipitridae,Nisaetus,nipalensis,"Hodgson, 1836",Mountain Hawk-eagle,biological,3be3f820-1462-4a04-ae0d-bce98d6cf08b,Mountain Hawk-eagle
58,2003056,Mammalia,Cetartiodactyla,Cervidae,Muntiacus,vaginalis,"Boddaert, 1785",Northern Red Muntjac,biological,9e90f0ba-0009-490b-b975-55829245a28e,Northern Red Muntjac
107,2015309,Aves,Passeriformes,Muscicapidae,Brachypteryx,leucophris,"Temminck, 1827",Lesser Shortwing,biological,a9e3b266-50dc-4759-80ca-8c7c972ee4f4,Lesser Shortwing


In [151]:
def list_labels(mega_result, ground_truth_table, THRESHOLD):
  """
  list actual labels and prediction labels
  """

  def has_object(detections, threshold, category_id):
    for i in range(len(detections)):
      if detections[i]['conf'] > threshold and detections[i]['category'] == category_id:
        return True
    return False

  print(len(mega_result['images']), "images")
  print('Generating actual and prediction lists...')

  # get list of files
  files = []
  for i in range(len(mega_result['images'])):
    files.append(mega_result['images'][i]['file'].split('/')[-1])

  category = {
      '1': 'Animal',
      '2': 'Human',
      '3': 'Vehicle'
  }

  y_actu = []
  y_pred = []

  for i in range(len(mega_result['images'])):
    try:
      # actual label
      actu = ground_truth_table[ground_truth_table['FileName'] == files[i]]['species_common_name'].iat[0]

      # prediction label
      # if the image has any animal
      if has_object(mega_result['images'][i]['detections'], THRESHOLD, '1'):
        if actu == '':
          y_pred.append('Animal')
          y_actu.append('Blank')
        else:
          y_pred.append('Animal')
          y_actu.append(actu)

      # if the image has any human
      if has_object(mega_result['images'][i]['detections'], THRESHOLD, '2'):
        y_pred.append('Human')
        if actu == '':
          y_actu.append('Blank')
        else:
          y_actu.append(actu)

      # if not have any human or animal
      if (not has_object(mega_result['images'][i]['detections'], THRESHOLD, '1')) and (not has_object(mega_result['images'][i]['detections'], THRESHOLD, '2')):
        y_pred.append('Blank')
        if actu == '':
          y_actu.append('Blank')
        else:
          y_actu.append(actu)    
    except:
      # print('File not found', files[i])
      pass
  
  y_pr = np.array(y_pred)
  print('Reduction rate', len(y_pr[y_pr == 'Blank']) / len(y_pr))
  return y_actu, y_pred


In [152]:
def create_confusion_matrix(y_actu, y_pred):
  print('Generating confusion matrix...')

  y_actu = pd.Series(y_actu, name='Actual')
  y_pred = pd.Series(y_pred, name='Prediction')
  df_confusion = pd.crosstab(y_pred, y_actu, rownames=['Prediction'], colnames=['Actual'], margins=True)

  # normalized confusion matrix using:
  df_conf_norm = round(df_confusion / df_confusion.sum(axis=1), 2)

  # delete nan columns
  df_conf_norm.dropna(axis = 1, inplace=True)

  # drop unecessary columns
  df_conf_norm.drop(columns=['All'], inplace=True)

  # collapse columns
  confusion_matrix = pd.DataFrame(index = ['Animal', 'Human', 'Blank', 'Total', 'Recall'])
  for col in df_confusion.columns:
    all = df_confusion[col]['All']
    animal = all - df_confusion[col]['Blank'] - df_confusion[col]['Human']
    human = df_confusion[col]['Human']
    blank = df_confusion[col]['Blank']
    total = animal + human + blank
    recall = 0
    # recall
    if col == 'Human':
      recall = human / total
    elif col == 'Blank':
      recall = blank / total
    else:
      recall = animal / total
    confusion_matrix[col] = [str(animal), str(human), str(blank), str(total), str(round(recall,2))]

  return confusion_matrix

In [153]:
def get_taxonomical_class(species_common_name, taxon_match_table=taxon_match_table):
  """
  get taxonomical class level of a species
  """
  row = taxon_match_table[taxon_match_table['FFI_species_name'] == species_common_name]
  try:
    taxon_class = row['class'].iat[0]
  except:
    print('     Can not find taxonomy info for', species_common_name)
  return taxon_class

# test
get_taxonomical_class('Bird')

'Aves'

In [154]:
def get_scientific_name(species_common_name, taxon_match_table=taxon_match_table):
  """
  get genus and species name of a species
  """
  row = taxon_match_table[taxon_match_table['FFI_species_name'] == species_common_name]
  try:
    genus, species = row['genus'].iat[0], row['species'].iat[0]
  except:
    print('     Can not find taxonomy info for', species_common_name)
  return genus, species

# test
get_scientific_name("Owston's Civet")

('Chrotogale', 'owstoni')

In [155]:
def get_conservation_status(genus, species):
  """
  get consevation status of a species from IUCN red list API
  https://apiv3.iucnredlist.org/api/v3/docs#species-history-name
  """
  api_token = "9bb4facb6d23f48efbf424bb05c0c1ef1cf6f468393bc745d42179ac4aca5fee"
  response = requests.get(f"https://apiv3.iucnredlist.org/api/v3/species/history/name/{genus}%20{species}?token={api_token}")
  try:
    status = response.json()['result'][0]['category']
  except:
    print('     Can not find conservation status for', genus, species)
  return status

# test
get_conservation_status('Chrotogale', 'owstoni')

'Endangered'

In [156]:
def visualize_recall_bargraph(taxon_class, confusion_matrix, THRESHOLD):
  """
  visualize recall values by bargraph by taxonomical class
  """

  print(f"Generating visualization plot for {taxon_class}...")
  total = []
  recall = []
  names = []

  image_count = len(mega_result['images'])

  data = confusion_matrix

  for species in data.columns[:-1]: # not include All 
    try:
      if get_taxonomical_class(species) in taxon_class:
        recall.append(data[species][4])
        total.append(data[species][3])
        names.append((species))
    except:
      pass

  recall_stats = pd.DataFrame({'name':names, 'recall':recall, 'total':total})
  recall_stats['recall'] = recall_stats['recall'].astype('float64')
  recall_stats['total'] = recall_stats['total'].astype('int')
  recall_stats.sort_values('recall', inplace=True, ascending=True)

  # creating the bar plot
  species_name = recall_stats['name'].to_list()
  values = recall_stats['recall'].to_list()
  total = recall_stats['total'].to_list()
  avg_recall = np.mean(recall_stats['recall'])

  colors = []
  color_map = {
               "Not Evaluated": "lightblue",
               "Data Deficient": "lightyellow",
               "Least Concern": "mediumseagreen",
               "Near Threatened": "olive",
               "Vulnerable": "orange",
               "Endangered": "orangered",
               "Critically Endangered": "maroon",
               "Extinct in the Wild": "black",
               "Extinct": "silver",
               "Unable to find status": "violet"
              }
  for name in species_name:
    try:
      genus, species = get_scientific_name(name)
      status = get_conservation_status(genus, species)
      colors.append(color_map[status])
    except:
      colors.append(color_map["Unable to find status"])

  fig = plt.figure(figsize=(10, 20))
    
  plt.barh(species_name, values, color=colors)
  for i, v in enumerate(values):
      plt.text(v, i, str(round(v,2)), color='blue', fontweight='bold')
      plt.text(v+0.07, i, f'{str(total[i])} images', color='blue', fontweight='bold')
  # for i, v in enumerate(total):

  plt.xlabel("Values")
  plt.ylabel("Species name")
  labels = list(color_map.keys())
  handles = [plt.Rectangle((0,0),1,1, color=color_map[label]) for label in labels]
  plt.legend(handles, labels)
  plt.title(f"Evaluate over {image_count} images \nclass: {taxon_class} \nthreshold:  {THRESHOLD} \naverage_recall_value: {round(avg_recall,2)}")
  # plt.savefig(f"123.png")
  plt.show()


In [157]:
def recall_evaluation(THRESHOLD, taxon_class):
  y_actu, y_pred = list_labels(mega_result, ground_truth_table, THRESHOLD)
  confusion_matrix = create_confusion_matrix(y_actu, y_pred)
  visualize_recall_bargraph(taxon_class, confusion_matrix, THRESHOLD)

In [ ]:
recall_evaluation(0.5, ['Mammalia', 'Insecta', 'Reptilia'])

110166 images
Generating actual and prediction lists...


In [ ]:
recall_evaluation(0.5, ['Aves'])